In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
from scipy.stats import linregress
from itertools import product
import math
from sklearn.linear_model import LinearRegression
from dataclasses import dataclass
from typing import Literal
from enum import Enum

pd.options.plotting.backend = "plotly"

In [ ]:
# https://docs.screeps.com/api/#Creep
CREEP_TICKS_LIFE = 1500
MAX_BODY_SIZE = 50
MAX_DISTANCE = 50

# https://docs.screeps.com/api/#StructureRoad
ROAD_ENERGY_REPAIR_RATIO = 0.01
ROAD_HEATH = 1000

ROOM_SOURCE_COUNT = 2
SOURCE_ENERGY_REGEN_TICKS = 300

BASE_ROAD_DECAY_RATE = 100


class Terrain(Enum):
    plain = 1.0
    road = 0.5


class SourceEnergy(Enum):
    owned = 3000
    reserved = 3000
    unreserved = 3000


In [ ]:
@dataclass
class Creep:
    work: int
    carry: int
    move: int

    @property
    def harvested_per_tick(self) -> int:
        return self.work * 2

    @property
    def cost(self) -> int:
        return self.work * 100 + self.carry * 50 + self.move * 50

    @property
    def size(self) -> int:
        return self.work + self.carry + self.move

    @property
    def capacity(self) -> int:
        return self.carry * 50

    @property
    def ticks_to_full(self) -> int:
        return self.capacity / self.harvested_per_tick

    def ticks_per_move(self, loaded: bool, terrain_type: str):
        move_requirement = self.move / Terrain[terrain_type].value

        # The game does not allow speeds greater than 1 move per tick
        if loaded:
            return math.ceil((self.work + self.carry) / move_requirement) or 1

        return math.ceil(self.work / move_requirement) or 1

    def round_trip(self, distance: int, terrain_type: str) -> float:
        forward = distance * self.ticks_per_move(True, terrain_type) + distance
        backward = distance * self.ticks_per_move(False, terrain_type) + distance

        return forward + backward

    def harvesting_round_trip(self, distance: int, terrain_type: str) -> float:
        return self.round_trip(distance, terrain_type) + self.ticks_to_full + 1


In [ ]:
@dataclass
class Room:
    # Creep model
    creep: Creep

    # Source distance from storage
    distance: int

    owned: bool = True

    source_count: int = 2

    @property
    def source_energy(self) -> int:
        source_type = "owned" if self.owned else "unreserved"
        return SourceEnergy[source_type].value

    @property
    def extractable_energy(self) -> float:
        extractable = (CREEP_TICKS_LIFE / SOURCE_ENERGY_REGEN_TICKS) * self.source_energy
        return extractable * self.source_count

    @property
    def road_repair_cost(self) -> float:
        """Repair costs per round trip"""
        road_decay_rate = BASE_ROAD_DECAY_RATE - self.creep.size * 2  # <= Round trip
        decay_cost_tick = (
            (CREEP_TICKS_LIFE / ROAD_HEATH) * road_decay_rate
        ) * ROAD_ENERGY_REPAIR_RATIO
        return decay_cost_tick * self.distance

    def profit(self, terrain_type: Terrain) -> float:
        round_trips = math.floor(
            CREEP_TICKS_LIFE / self.creep.harvesting_round_trip(self.distance, terrain_type)
        )
        profit = round_trips * self.creep.capacity - self.creep.cost

        if terrain_type == "road":
            profit = profit - self.road_repair_cost * round_trips

        return profit if profit < self.extractable_energy else self.extractable_energy

    def cost_benefit(self, terrain_type: str) -> float:
        return self.profit(terrain_type) / self.creep.cost

In [ ]:
class ProportionsRoads:
    @classmethod
    def cost_benefit_plots(cls, terrain_type):
        distances = [*range(5, MAX_DISTANCE, 1)]

        work_parts = [*range(1, MAX_BODY_SIZE)]
        carry_parts = [*range(1, MAX_BODY_SIZE)]
        move_parts = [*range(1, MAX_BODY_SIZE)]

        # Body combinations with less than MAX_BODY_SIZE parts
        parts_prod = filter(
            lambda x: (x[0] + x[1] + x[2]) < MAX_BODY_SIZE,
            product(work_parts, carry_parts, move_parts),
        )
        parts_prod = filter(lambda x: Creep(*x).ticks_per_move(False, terrain_type) < 5, parts_prod)
        # parts_prod

        mp = lambda w: [str(w), *w[0], w[1], Room(Creep(*w[0]), w[1]).cost_benefit(terrain_type)]
        cost_benefit_heatmap = list(map(mp, product(parts_prod, distances)))

        df = pd.DataFrame(
            cost_benefit_heatmap,
            columns=[
                "label",
                "work_parts",
                "carry_parts",
                "walk_parts",
                "distance",
                "cost_benefit",
            ],
        )
        df = df[df["cost_benefit"] > 1]
        df["cost_benefit"] = round(df["cost_benefit"], 4)
        # print(df.groupby('distance').apply(lambda x: x.nlargest(5, 'cost_benefit')))

        df_reg = df.groupby("distance").apply(lambda x: x.nlargest(2, "cost_benefit")).copy()
        df_reg = df_reg.reset_index(drop=True).sort_values(
            ["distance", "cost_benefit"], ascending=False
        )
        df_reg["ticks_per_move_loaded"] = (df_reg["work_parts"] + df_reg["carry_parts"]) / df_reg[
            "walk_parts"
        ]

        df_norm = df_reg.copy()

        df_norm["total_parts"] = (
            df_norm["work_parts"] + df_norm["carry_parts"] + df_norm["walk_parts"]
        )
        df_norm["work_parts"] = df_norm["work_parts"] / df_norm["total_parts"] * 100
        df_norm["carry_parts"] = df_norm["carry_parts"] / df_norm["total_parts"] * 100
        df_norm["walk_parts"] = df_norm["walk_parts"] / df_norm["total_parts"] * 100

        df_norm.plot(x="distance", y=["work_parts", "carry_parts", "walk_parts"])

        fig = px.scatter(df_norm, x="distance", y=["work_parts", "carry_parts", "walk_parts"])

        return df_norm, fig

In [ ]:
df_norm, fig = ProportionsRoads.cost_benefit_plots(terrain_type="plain")
fig.show()

In [ ]:
df_norm

In [ ]:
df_norm, fig = ProportionsRoads.cost_benefit_plots(terrain_type="road")
fig.show()